# Exercise 2

The goal of the exercise is to implement the MPPI algorithm discussed in class to control a (simplified) 3 degrees of freedom manipulator.

You do not have access to the dynamics of the manipulator but only to a simulator (the ``simulate`` function) which you can use to test how the manipulator would behave for a given control sequence.
* The states $x$ of the manipulator are the angles (in radians) of the 3 joints.
* The controls $u$ set the velocity of the joints (3 controls).
* The tip of the manipulator is $p(x)$ and depends on the current states $x$ (this is computed using a ``forward_kinematics`` function).
* We assume a single shooting problem where we only optimize the control sequence $\bar{u} = u_0,u_1,\cdots$ and compute the resulting states using the simulator

The goal is to get the manipulator's tip $p(x)$ to reach a desired goal position by minimizing (with respect to $\bar{u}$)
$$l(\bar{u}) = \sum_{n=0}^{N_{horizon}} (p(x_n) - p_{desired})^2 + 0.01 u_n^2$$

To do so, you will implement a model-predictive controller using the MPPI algorithm in its inner loop.

## MPPI algorithm
At each iteration, the MPPI algorithm computes an improved control through
$$\bar{u} \leftarrow \bar{u} - \alpha \frac{ - \lambda \mathbb E \left[  \exp\left(- \frac{1}{\lambda} l(\bar{u} + \mu \epsilon) \right) \Sigma^{-1}  \epsilon \right]}{ \mu \mathbb   E \left[ \exp\left(- \frac{1}{\lambda} l(\bar{u} + \mu \epsilon) \right) \right]  }$$

which is approximated through Monte-Carlo sampling by
$$\bar{u} \leftarrow \bar{u} - \alpha \frac{ - \lambda \mathbb \sum_{k=1}^K \left(\exp\left(- \frac{1}{\lambda} l(\bar{u} + \mu \epsilon_k) \right) \Sigma^{-1}  \epsilon_k \right)}{ \mu \mathbb   \sum_{k=1}^K \left( \exp\left(- \frac{1}{\lambda} l(\bar{u} + \mu \epsilon_k) \right) \right)  }$$


Tips to implement the algorithm:
* The parameters $\alpha = 1$, $\Sigma = I$, $\lambda=0.1$, $\mu=2$ and $K=50$ work well for this problem (use the same samples to evaluate both sums)
* Do only one update of $\bar{u}$ per control cycle (i.e. one descent step) and reuse the previous trajectory as an initial guess shifted by one time-step (for the warm start)
* Use $N_{horizon} = 10$ (optimize 10 time steps ahead)
* Use the numpy function ``random.multivariate_normal`` to draw a random sample from a multi-dimensional Gaussian distribution (you will need to draw $3N_{horizon}$ samples for $\epsilon$)

## Questions
1. Implement the MPPI algorithm and use it to reach the following positions $(2,1)$, $(2,2)$, $(1,-2)$ (one full run per position). Run the simulation for 500 time steps.
2. How does the algorithm perform across different runs? Do you always get the same result? Why?
3. Plot the average cost and its variance across 10 runs for each target position

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import manipulator

In [2]:
## the manipulator package contains all the necessary functions to simulate the robot

## the forward_kinematics function returns the position of the tip of the manipulator as a function of the joint positions
joint_position = np.array([0.,0.,0.])
tip_position = manipulator.forward_kinematics(joint_position)
print(f'the tip of the manipulator is at x={tip_position[0]:0.1f}, y={tip_position[1]:0.1f} ' + \
      f'when the joints are at {joint_position[0]:0.1f}, {joint_position[2]:0.1f}, {joint_position[2]:0.1f}')

joint_position = np.array([0.5,0.5,0.5])
tip_position = manipulator.forward_kinematics(joint_position)
print(f'the tip of the manipulator is at x={tip_position[0]:0.1f}, y={tip_position[1]:0.1f} ' + \
      f'when the joints are at {joint_position[0]:0.1f}, {joint_position[2]:0.1f}, {joint_position[2]:0.1f}')

the tip of the manipulator is at x=2.4, y=0.0 when the joints are at 0.0, 0.0, 0.0
the tip of the manipulator is at x=1.4, y=1.8 when the joints are at 0.5, 0.5, 0.5


In [11]:
## the manipulator can be simulated with a control trajectory of length N

## the simulate function returns the time, state trajectory [3,N] and control trajectory [3,N]

N = 500 #number of timesteps
initial_state = np.array([0.,0.,0.])

## we compute a random control trajectory drawn from a multivariate normal distribution
u = np.reshape(np.random.multivariate_normal(np.zeros(3*N), np.eye(3*N)),[3,N])
t,x, u = manipulator.simulate(initial_state, u, N)

## the states can be displayed using the animate_robot function as follows
manipulator.animate_robot(x)

Your browser does not support the video tag.

In [134]:
!pip install memory-profiler
%load_ext memory_profiler

You should consider upgrading via the 'c:\users\dell\anaconda3\python.exe -m pip install --upgrade pip' command.


In [273]:
## fill in the controller function
# u=np.zeros([3,10])
# pd=np.array([2,2])
# st=[]

# def controller(states,samples=50,alpha=1,lamb=.1,mu=2,N_timesteps=500):
#     ''' simplistic MPC loop '''
#     global u,pd
#     #shifting 1 timestep
    
#     # control
#     # u[:,:-1]=u[:,1:]
#     # u[:,-1] =0
#     Nhorizon = 10
#     # ep=np.reshape(np.random.multivariate_normal(np.zeros(samples*3*Nhorizon), np.eye(samples*3*Nhorizon)),[samples,3,Nhorizon])
#     ep=np.random.randn(samples, 3, Nhorizon) #this is same as using the above function in a loop 50 times!
#     u_guess = u+mu*ep
    
#     # states=states[:,10]
#     sim= lambda x:manipulator.simulate(states, x, 10)
#     stt=[]
#     for i in range(ep.shape[0]):
#         print()
#         t, st, control= manipulator.simulate(states,u_guess[i] , 10)  #np.apply_along_axis(sim, axis=1, arr=u_guess)
#         stt.append(s)
#     states=np.array(stt)
#     print(states.shape)
#     # u=control
#     # states=states[:,1]
#     # u=control[:,1]
#     l=lambda x: np.sum(np.square(manipulator.forward_kinematics(x)-pd))
#     la= np.apply_along_axis(l, axis=1, arr=states)
#     if la.ndim > 0:
#         la=np.sum(la[:min(10,len(la))])

#     contr = lambda x: u_guess
    

#     l= la+0.01*np.sum(np.square(u_guess),axis=(1, 2), keepdims=True)
#     exp =  np.exp((-1/lamb)*l)
#     # print(np.max(l),np.min(l))
#     num=np.sum(-lamb*exp*ep,axis=0)
#     den=np.sum(mu*exp,axis=0)
    
#     print(num/den)
#     u = u - alpha*num/den
#     print(u)
#     return u[:,0]


# ## run the MPC controller
# u=np.zeros([3,10])
# pd=np.array([2,2])

# controller(np.zeros([3,]))


In [441]:
u=np.zeros([3,10])
pd=np.array([2,2])
def controller(states,Nhorizon=10,samples=50,alpha=1,lamb=.1,mu=2):
    ''' simplistic MPC loop '''
    global pd,u

    num=0
    den=0
    u[:,:-1]=u[:,1:]
    u[:,-1]=0
    all_costs = np.zeros(samples)
    all_perturbations = np.zeros((samples, 3, Nhorizon))
    for i in range(samples):
        ep = np.reshape(np.random.multivariate_normal(np.zeros(3*Nhorizon), np.eye(3*Nhorizon)),[3,Nhorizon])
        # ep = np.random.randn(3, Nhorizon) #faster for simple variance
        u_guess = u+mu*ep
        t, st, control = manipulator.simulate(states,u_guess , Nhorizon)
        # print(st.shape)
        l=lambda x: np.sum(np.square(manipulator.forward_kinematics(x)-pd))
        la= np.sum(np.apply_along_axis(l, axis=0, arr=st))
        
        
        l= la+0.01*np.sum(np.square(u_guess))
        all_costs[i] = l
        all_perturbations[i] = ep
        # print(la,l-la)
        # if np.isnan(l)
            # print(l,la)
        
        # exp =  np.exp((-1/lamb)*l)
        # # print(np.sum(np.square(u_guess)).shape,la.shape)
        # # print(exp)
        # num+=-lamb*exp*ep
        # den+=mu*exp +1e-25 #added to avoid zero divide
    min_cost = np.min(all_costs)
    
    # Subtract min_cost *before* exponentiating. This is the trick!
    # The (S - min_cost) term will be <= 0, so exp() is between (0, 1]
    stable_costs = all_costs - min_cost 
    
    # Calculate weights. 'exp' is now a stable array of shape (samples,)
    exp_weights = np.exp((-1 / lamb) * stable_costs)

    # Normalize the weights
    sum_weights = np.sum(exp_weights) + 1e-25  # Avoid zero divide
    weights = exp_weights / sum_weights
    delta_u = -1*(lamb/mu)* np.sum(weights[:, np.newaxis, np.newaxis] * all_perturbations, axis=0)
        # if den < 1e-20:
        #     print(l,la,u_guess)
    # print(num/den)
    u = u - alpha* delta_u#num/den #
    
    
    # u[:,-1] =u[:,-2]
    return u[:,0]

In [251]:
 guo8himj9,k0il.-;=''

SyntaxError: invalid syntax (<ipython-input-251-2466a23a8241>, line 1)

In [307]:

ep =  ep=np.random.randn(3, 10) #faster for simple variance
u_guess = 7*np.ones((3, 10)) +2*ep
t, st, control = manipulator.simulate(np.zeros([3,]),u_guess , 10)
pd=np.array([2,2])
l=lambda x: np.sum(np.square(manipulator.forward_kinematics(x)-pd))
kk=lambda x: np.square(manipulator.forward_kinematics(x)-pd)
la= np.sum(np.apply_along_axis(l, axis=0, arr=st))
kd=np.apply_along_axis(kk, axis=0, arr=st)

16.120113260609948

1.8048513878454153e-35

In [442]:
u=np.zeros((3, 10))  #
pd=np.array([1,-2])

N_timesteps = 500
t, states, control = manipulator.simulate(np.zeros([3,]), controller, N_timesteps)

## display the results
manipulator.animate_robot(states)

## Note that the simulate function can take either a control trajectory of size [3,N] 
## or a function f(states) that will be called at each control cycle (cf. examples above)

Your browser does not support the video tag.